In [1]:
import numpy as np
import matplotlib.pyplot as plt
import dwave_networkx as dnx
import networkx as nx
%matplotlib inline
import matplotlib as mpl
import math
from dwave.system import DWaveSampler
import minorminer as mm
import dwave.inspector

In [2]:
sampler = DWaveSampler(solver=dict(topology__type='pegasus'),
                       token="DEV-5fa1054cd25ed5a94ead77ab54c60e280f772f28")

print("QPU {} was selected.".format(sampler.solver.name))

QPU Advantage_system6.1 was selected.


In [8]:
from notebook import notebookapp
servers = list(notebookapp.list_running_servers())
print(servers)

[{'base_url': '/', 'hostname': 'localhost', 'notebook_dir': '/Users/prathunarasimhan', 'password': False, 'pid': 2143, 'port': 8888, 'secure': False, 'sock': '', 'token': '56441649e3e5b6b9d26c73fe13e845d4e99b6328e54975ad', 'url': 'http://localhost:8888/'}]


In [3]:
#create a target graph
T=nx.Graph()
T.add_edges_from(sampler.edgelist)
T.add_nodes_from(sampler.nodelist)

In [4]:
#create a source graph
S=nx.Graph()
#spin chain length
N=2000
#define the nodes
S.add_nodes_from(list(range(N)))
spins=list(S.nodes)
#define the edges. the first node is also the last node (PBC)
for i in range(len(spins)-1):
    S.add_edge(spins[i],spins[i+1])
S.add_edge(spins[len(spins)-1],spins[0])

In [5]:
cond=0
count=0

while (cond==0 and count<1000):

    embedding=mm.find_embedding(S,T,verbose=1)

    biggerthan1=[]
    for value in embedding.values():
        if len(value)>1:
            biggerthan1.append(value)
    
    if len(biggerthan1)>0:
        cond=0
    if len(biggerthan1)==0:
        cond=1
        
    if count==1000:
        print("reached max interations")
        
    count+=1

print(embedding)

{0: [4259], 1: [2707], 1999: [2586], 2: [4184], 3: [2781], 4: [4169], 5: [4154], 6: [2706], 7: [4004], 8: [2690], 9: [3974], 10: [3973], 11: [3972], 12: [2375], 13: [3853], 14: [2540], 15: [3838], 16: [2434], 17: [2435], 18: [4092], 19: [2391], 20: [2390], 21: [3957], 22: [2420], 23: [4078], 24: [2481], 25: [2482], 26: [4272], 27: [2421], 28: [4213], 29: [2451], 30: [4168], 31: [2571], 32: [4079], 33: [2646], 34: [4064], 35: [2691], 36: [4094], 37: [2841], 38: [4124], 39: [2751], 40: [4109], 41: [2765], 42: [3944], 43: [2840], 44: [3929], 45: [2764], 46: [3839], 47: [3824], 48: [2735], 49: [2750], 50: [2749], 51: [3809], 52: [2659], 53: [3778], 54: [2554], 55: [3748], 56: [2509], 57: [3868], 58: [2494], 59: [3793], 60: [2599], 61: [3869], 62: [2570], 63: [3913], 64: [3928], 65: [2614], 66: [2615], 67: [2616], 68: [2617], 69: [4394], 70: [2708], 71: [4484], 72: [2843], 73: [2842], 74: [2827], 75: [2826], 76: [2825], 77: [2824], 78: [2839], 79: [3764], 80: [3779], 81: [2779], 82: [3914],

In [6]:
couplers={}
for edge in list(S.edges):
    spin0=edge[0]
    spin1=edge[1]
    qubit0=embedding.get(spin0)[0]
    qubit1=embedding.get(spin1)[0]
    couplers.update({(qubit0,qubit1):-1})

In [7]:
response=sampler.sample_ising(h={},J=couplers,annealing_time=1,num_reads=10)
print(response)
dwave.inspector.show(response)

  30 31 33 34 35 38 39 40 45 47 48 49 50 55 57 58 59 ... 5728  energy num_oc.
0 -1 -1 +1 -1 -1 -1 -1 -1 -1 -1 +1 +1 +1 -1 -1 -1 -1 ...   +1 -1876.0       1
1 -1 -1 -1 -1 -1 +1 +1 +1 -1 +1 +1 +1 +1 +1 -1 -1 -1 ...   -1 -1864.0       1
2 -1 -1 +1 +1 +1 +1 +1 +1 -1 -1 -1 -1 -1 +1 -1 +1 +1 ...   -1 -1852.0       1
3 -1 -1 -1 -1 -1 +1 +1 +1 +1 +1 -1 -1 -1 +1 -1 -1 +1 ...   -1 -1848.0       1
4 +1 +1 +1 +1 +1 -1 +1 +1 +1 +1 +1 +1 +1 +1 -1 -1 -1 ...   +1 -1848.0       1
5 -1 -1 -1 -1 -1 -1 -1 -1 -1 +1 +1 +1 +1 -1 +1 +1 -1 ...   -1 -1848.0       1
6 -1 -1 -1 -1 -1 -1 +1 +1 -1 +1 +1 +1 +1 +1 -1 -1 -1 ...   +1 -1844.0       1
7 +1 +1 +1 +1 +1 +1 +1 +1 +1 +1 +1 +1 +1 -1 -1 -1 -1 ...   -1 -1836.0       1
8 +1 +1 +1 +1 +1 +1 +1 +1 -1 +1 +1 +1 +1 -1 -1 +1 +1 ...   -1 -1836.0       1
9 -1 -1 -1 -1 -1 +1 -1 -1 -1 -1 +1 +1 +1 -1 -1 -1 -1 ...   +1 -1824.0       1
['SPIN', 10 rows, 10 samples, 2000 variables]


'http://127.0.0.1:18000/?problemId=8e521675-956b-4cb5-8cc0-83ae5ad6cf9d'